## Do the thing

#### test one country

In [1]:
from run_multiprocessing import run_multiprocessing
from multiprocessing import Pool
from functools import partial
import numpy as np

import sys
from pandas import Series,DataFrame,read_csv
import os
from country_run import *
from lib_for_data_reading import *
from lib_for_cc import *
import re
import datetime

In [ ]:
if __name__ ==  '__main__': 
    ######################################
    # Set parameters
    #
    nameofthisround = 'spring2020'
    #
    data2day      = 30
    year          = 2030
    povline       = 1.9
    ini_year      = 2011 #replace with get_ini_year() below
    
    ######################################
    # Set directories
    #
    # INPUT directories
    model             = os.getcwd()
    model             = model.replace('\\','/')
    data              = model+'/data/'
    scenar_folder     = model+'/scenar_def/'
    finalhhdataframes = model+'/finalhhdataframes/'
    ssp_folder        = model+'/ssp_data/'
    iiasa_data        = model+'/iiasa_data/'
    
    ######################################
    # Load necessary files
    #
    #load climate change impacts data 
    price_increase_reg = read_csv(iiasa_data+"boundaries_food2.csv")
    food_share_data = read_csv(data+"food_shares_wbreg.csv")
    #
    ####
    # Now WITH malaria!!!
    malaria_diff    = read_csv(data+"malaria_diff_2006-2035.csv")
    malaria_bounds  = read_csv(data+"malaria_bounds.csv",index_col="var")
    diarrhea_bounds = read_csv(data+"diarrhea_bounds.csv",index_col="var")
    #
    #####
    diarrhea_shares  = read_csv(data+"diarrhea_who.csv",index_col="Region")
    diarrhea_shares.index=diarrhea_shares.index.str.strip()
    diarrhea_reg    = read_csv(data+"wbccodes2014_who.csv",index_col="country")
    #
    #####
    # No natural disasters!
    fa                       = read_csv(data+"disasters/av_fa_for_cc_model.csv",index_col='iso3')
    vulnerabilities          = read_csv(data+"disasters/vulnerabilities.csv",index_col='iso3')
    #
    #####
    # Need to see what these are, as distinct from price_increase. Probably temperature?
    climate_param_bounds     = read_csv(data+"climate_param_bounds.csv",index_col="var")
    climate_param_thresholds = read_csv(data+"climate_param_thresholds.csv",index_col="var")
    #
    #
    # No Stunting!
    shares_stunting = None #read_csv(data+"share_stunting_who.csv")
    
    ######################################
    # Storage of results
    baselines  = "{}/baselines_{}/".format(model,nameofthisround)
    with_cc    = "{}/with_cc_{}/".format(model,nameofthisround)

    if not os.path.exists(baselines):
        os.makedirs(baselines)
    if not os.path.exists(with_cc):
        os.makedirs(with_cc)
        
    ######################################
    # Get list of available countries...
    list_csv=os.listdir(finalhhdataframes)
    all_surveys=dict()
    for myfile in [_ for _ in list_csv if 'DS_Store' not in _ and _ != '_Final_CPI_PPP_to_be_used.csv']:
        if 'finalhhframe' in myfile:
            cc = re.search('(.*)_finalhhframe.csv', myfile).group(1)
            all_surveys[cc]=read_csv(finalhhdataframes+myfile)

    ####
    #load codes, population
    codes         = read_csv('wbccodes2014.csv')
    codes_tables  = read_csv(ssp_folder+"ISO3166_and_R32.csv")
    ssp_pop       = read_csv(ssp_folder+"SspDb_country_data_2013-06-12.csv",low_memory=False)
    ssp_gdp       = read_csv(ssp_folder+"SspDb_compare_regions_2013-06-12.csv")
    hhcat         = read_csv(scenar_folder+"hhcat_v2.csv")
    industry_list = read_csv(scenar_folder+"list_industries.csv")

    ####
    # Drivers of scenarios
    uncertainties    = ['shareag','sharemanu','shareemp','shareurban','grserv','grag','grmanu','skillpserv',
                        'skillpag','skillpmanu','p','b','voice']
    lhssample        = read_csv(scenar_folder+"lhs-table-500-13uncertainties.csv")
    ranges = DataFrame(columns=['min','max'],index=uncertainties)

    impact_scenars = read_csv(scenar_folder+"impact-scenarios-low-high.csv")
    
    ######################################
    # Packaging for country_run()
    #
    datalist=(hhcat, #hh category and its attributes (isserv, ismanu, isag, skilled)
              industry_list, #classification of industries into serv, ag, manu
              codes_tables, #country name - country codes, R32 region classification
              lhssample, #experimental design, consisting of uncertain parameters
              ranges, #min and max range of experimental design
              #ASKING: difference between ssp_pop and ssp_gdp?
              ssp_pop, #regional population projection 2010-2100 based on different SSPs (SSP1-SSP5)
              ssp_gdp, #regional gdf (population?) projection 2010-2100 based on different SSPs (SSP1-SSP5)
              #ASKING: what is consSeg and food_share in food_share_data?
              food_share_data, #regional food share data
              #ASKING: what is impact_scenars? the low/high impact?
              impact_scenars, #impacts: food_cursor, temp_impact, paramshstunt, malarua_cursor, diarrhea_cursor, disasters_cursor
              #ASKING: what is prodd, priced, pqd?
              price_increase_reg, #minimum and maximum price increase for each region, for SSP4 and 5
              malaria_bounds, #minimum and maximum lostdays and eventcost due to malaria
              diarrhea_bounds, #minimum and maximum lostdays and eventcost due to diarrhea
              shares_stunting, #empty for now
              #ASKING: what is diff 2020s?
              malaria_diff, #2020s difference of malaria for 145 countries
              diarrhea_shares, #percentage of diarrhea population in each region
              diarrhea_reg, #region code conversion between wb and who
              climate_param_bounds, #min and max values for climate parameters
              #ASKING: what is th_nostunt and th_diarrhea
              climate_param_thresholds, #threshold for stunting and diarrhea
              codes, #countries and regions codes
              #ASKING: what is fa?
              fa, #hazard database
              vulnerabilities)
    #
    ssp_to_test   = [4,5]
    #

    paramvar=(year,ini_year,data2day,ssp_to_test,povline)
    switches = ['all']
    
    ######################################
    # multiprocessing
    #
    print('starting multiprocessing')
    scenarios = list(np.arange(0,10,1))
    num_processors = 4
    p=Pool(processes=num_processors)
    output = p.map(partial(run_multiprocessing,datalist=datalist,paramvar=paramvar,all_surveys=all_surveys,
                           switches=switches,baselines=baselines,with_cc=with_cc), scenarios)
    print(output)

starting multiprocessing


In [2]:
#read some results
list_csv = os.listdir(os.getcwd()+'/baselines_spring2020/')
list_csv = [x for x in list_csv if 'BGD' in x]

df = pd.DataFrame()
for csv in list_csv:
    df = df.append(pd.read_csv(os.getcwd()+'/baselines_spring2020/'+csv))
len(df)

20

In [4]:
df.sort_values(['ssp', 'scenar'])

,Unnamed: 0,country,year,scenar,ssp,avg_income_bott20,avg_income,avg_income_urban,avg_income_bott20_urban,avg_income_bott20_rural,...,grmanu,skillpserv,skillpag,skillpmanu,p,b,voice,prod_gr_ag,prod_gr_serv,prod_gr_manu
0,0,BGD,2030,0,ssp4,1.773860,4.108454,4.989577,2.093195,1.674256,...,0.008222,1.543608,1.334268,1.073405,0.037707,0.0,0.352170,-0.027698,-0.017841,0.006184
0,0,BGD,2030,1,ssp4,1.826625,4.091911,4.778264,2.048663,1.749672,...,-0.013617,1.543608,1.334268,1.073405,0.114286,0.0,0.553172,-0.016072,-0.004974,-0.019897
0,0,BGD,2030,2,ssp4,2.062280,5.238006,6.211232,2.310021,1.966676,...,-0.013127,1.543608,1.334268,1.073405,0.004033,0.0,0.591478,0.020232,0.010980,-0.013336
0,0,BGD,2030,3,ssp4,2.191051,5.037501,5.683655,2.497729,2.087272,...,0.015384,1.543608,1.334268,1.073405,0.199197,0.0,0.525678,-0.008390,-0.009417,0.003582
0,0,BGD,2030,4,ssp4,1.997942,5.040488,5.838775,2.438121,1.872510,...,0.010622,1.543608,1.334268,1.073405,0.180914,0.0,0.346619,0.011847,-0.010093,0.000062
0,0,BGD,2030,5,ssp4,1.647187,3.988590,4.707334,1.892408,1.568135,...,-0.019198,1.543608,1.334268,1.073405,0.176939,0.0,0.035610,-0.033928,-0.028988,-0.029198
0,0,BGD,2030,6,ssp4,2.035110,5.227061,6.002162,2.082007,2.008341,...,-0.025874,1.543608,1.334268,1.073405,0.067513,0.0,0.813156,0.021643,0.007255,-0.029451
0,0,BGD,2030,7,ssp4,2.016665,5.105347,6.662521,2.874023,1.853317,...,0.024401,1.543608,1.334268,1.073405,0.035049,0.0,0.929911,0.009359,-0.008378,0.022479
0,0,BGD,2030,8,ssp4,2.093173,4.870786,5.529643,2.279634,1.981105,...,-0.006569,1.543608,1.334268,1.073405,0.088147,0.0,0.106585,0.001918,-0.000739,-0.011382
0,0,BGD,2030,9,ssp4,1.923148,4.225920,4.752505,2.042662,1.880859,...,-0.014546,1.543608,1.334268,1.073405,0.172049,0.0,0.728291,-0.023622,0.009816,-0.024290
